In [10]:
import requests
from bs4 import BeautifulSoup

### Récupération des numéros d'articles du droit des contrats

In [11]:
url_contrat = "https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000006070721/LEGISCTA000032006712/"
response = requests.get(url_contrat)
soup = BeautifulSoup(response.text, 'html.parser')

codes = []
for a in soup.find_all('p', class_='name-article'):
    article = a.find('a')
    if article:
        codes.append(article.text.split(' ')[1])

In [12]:
print(codes[:10])

['1101', '1102', '1103', '1104', '1105', '1106', '1107', '1108', '1109', '1110']


### Récupération des liens des décisions

In [13]:
base_url = 'https://www.legifrance.gouv.fr/'
page_number = 1000
urls = {}

for page in range(1, page_number + 1):
    filter_url = base_url + 'search/juri?tab_selection=juri&searchField=ALL&searchType=ALL&dateDecision=01%2F01%2F2021+%3E+01%2F01%2F2024&cassFormation=CHAMBRE_CIVILE&cassFormation=CHAMBRE_CIVILE_1&cassFormation=CHAMBRE_CIVILE_2&cassFormation=CHAMBRE_CIVILE_3&typePagination=DEFAULT&sortValue=DATE_DESC&pageSize=10&page={}&tab_selection=juri'.format(page)
    response = requests.get(filter_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article', class_='result-item')
    for i, article in enumerate(articles):
        number_article = (page - 1) * 10 + i + 1
        url = base_url + article.find('a')['href']
        urls[number_article] = url

### Récupération du texte provenant du lien de la décision

In [14]:
import re

texts = {}
for number_article, url in urls.items():
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error {response.status_code} for {url}")
        continue
    soup = BeautifulSoup(response.text, 'html.parser')
    content = str(soup.find('div', class_='content-page').find('div', class_=None))
    text = content.replace('<div>', '').replace('</div>', '').replace('<br>', '\n').replace('<br/>', '\n')
    if soup.find('h1', class_='main-title'):
        titre = soup.find('h1', class_='main-title').text
    elif soup.find('h1', class_='main-title-light'):
        titre = soup.find('h1', class_='main-title-light').text
    else:
        titre = ""
    texts[number_article] = {
        "url": url,
        "titre": titre,
        "texte": text
    }

### Récupération des numéros d'articles à partir des textes

In [15]:
import re

text = (
    "Voici plusieurs exemples de texte contenant diverses mentions d'articles,"
    "tels que l'article 1101 du code civil,"
    "l'article L. 1181-1 du code rural,"
    "l'article 1102 du code civil, "
    "l'article 1103 du code civil,"
    "les articles 1111, 1111-1, 1112 et 1112-1 du code civil "
    "l'article 1113 du code Civil."
    "l'article 1114 de l'ancien code civil"
    "l'ancien article 1115 du code civil"
    "l'ancien article 1116 de l'ancien code civil"
    "les anciens articles 1117, 1118 et 1119 du code civil"
    "les anciens articles 1120, 1121 et 1122 de l'ancien code civil"
)


def extract_article_numbers(text):
    pattern = r"((?:l'?ancien\s|les anciens\s)?article[s]?\s+[\d, -]+(?:et [\d, -]+)?\s+(?:du|de l'?ancien)\s+code civil)"
    
    matches = re.findall(pattern, text, re.IGNORECASE)

    articles = []
    for block in matches:
        prefix = "ancien " if "ancien" in block.lower() else ""
        
        numbers = re.findall(r"(\d+(?:-\d+)?)", block)
        
        for num in numbers:
            if num in codes:
                article = f"{prefix}{num.strip()}"
                articles.append(article)

    return sorted(list(set(articles)), key=lambda x: (len(x), x))

In [16]:
for number, texte in texts.items():
    articles = extract_article_numbers(texte["texte"])
    texts[number]["articles_cités"] = articles

In [17]:
{k: v["articles_cités"] for k, v in texts.items() if v["articles_cités"]}

{8: ['1103'],
 12: ['1103', '1104'],
 17: ['1134', '1143'],
 20: ['1134'],
 29: ['1116'],
 32: ['1165'],
 57: ['1116'],
 72: ['1231-1'],
 84: ['1178'],
 85: ['1130', '1137', '1112-1'],
 91: ['1134'],
 107: ['1103', '1104'],
 110: ['1221'],
 128: ['1109'],
 132: ['1154'],
 134: ['1103', '1231-1'],
 136: ['1134'],
 137: ['1134', '1147', '1176'],
 139: ['1101', '1113', '1192'],
 141: ['1134', '1184'],
 147: ['1103'],
 158: ['1147'],
 181: ['ancien 1134'],
 231: ['1134'],
 267: ['1103', '1134'],
 268: ['1103'],
 269: ['1182'],
 296: ['1134'],
 303: ['1110', '1134'],
 328: ['1214', '1215'],
 349: ['1103'],
 362: ['1103'],
 366: ['1103'],
 367: ['1103'],
 382: ['1103'],
 392: ['1134'],
 397: ['1147'],
 398: ['1134', '1156'],
 402: ['1134'],
 439: ['1231-6'],
 478: ['1131', 'ancien 1131'],
 514: ['1149'],
 515: ['1103', '1134'],
 564: ['1134'],
 565: ['1147'],
 585: ['1184'],
 587: ['1165'],
 634: ['1131', 'ancien 1131'],
 635: ['1131', 'ancien 1131'],
 637: ['1104'],
 645: ['1134'],
 657: ['

### Analyse des numéros d'articles par année

In [19]:
import pandas as pd

df = pd.DataFrame(texts).T.explode('articles_cités')
df = df[df['articles_cités'].notnull()]

pattern = r"\b(\d{1,2})\s(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s(\d{4})\b"
df[["jour", "mois", "année"]] = df.titre.str.extract(pattern)

In [20]:
df[df.titre == df.iloc[434].titre]

,url,titre,texte,articles_cités,jour,mois,année
2755,https://www.legifrance.gouv.fr//juri/id/JURITE...,"Cour de cassation, civile, Chambre civile 2, 1...","LA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVILE,...",1134,15,décembre,2022
2755,https://www.legifrance.gouv.fr//juri/id/JURITE...,"Cour de cassation, civile, Chambre civile 2, 1...","LA COUR DE CASSATION, DEUXIÈME CHAMBRE CIVILE,...",ancien 1134,15,décembre,2022


In [21]:
df_citations = df[['année', 'articles_cités']].groupby(['année', 'articles_cités']).size().fillna(0).astype(int).to_frame('nombre_de_citations').reset_index()
top10_par_année = df_citations.groupby(['année']).apply(lambda x: x.sort_values(by='nombre_de_citations', ascending=False).head(10), include_groups=False)
top10_par_année = top10_par_année.groupby(['année', 'articles_cités']).nombre_de_citations.sum().to_frame('nombre_de_citations').sort_values(['année', 'nombre_de_citations'], ascending=False)
top10_par_année

nombre_de_citations
année articles_cités                     
2023  1134                             96
      1147                             66
      1103                             51
      1231-1                           14
      1131                             13
      1184                             13
      1104                              8
      1101                              7
      1116                              7
      1154                              7
2022  1134                            188
      1147                            160
      1103                             53
      1184                             35
      1116                             34
      1231-1                           29
      1154                             19
      1165                             17
      1131                             13
      1192                             12
2021  1134                            250
      1147                            171
      1103                             65
      1184                             42
      1116                             27
      1154                             26
      1156                             26
      1161                             22
      1108                             18
      1104                             15